# Databricks Notebook: Módulo 1 - Ingesta de Datos

In [0]:
# 1. Leer CSV: loan_applications.csv y transactions.csv
loan_df = (spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("/Volumes/workspace/default/workshop/loan_applications.csv")
)
trans_df = (spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("/Volumes/workspace/default/workshop/transactions.csv")
)

In [0]:
loan_df.display()

In [0]:
trans_df.display()

In [0]:
loan_df.printSchema()
trans_df.printSchema()

# Databricks Notebook: Módulo 2 - Exploración y Limpieza

In [0]:
# 1. Estadísticas básicas
# describe() ofrece count, mean, stddev, min, max.

# summary('min', '25%', '50%', '75%', 'max') para percentiles.

# approxQuantile('col', [0.25,0.5,0.75], 0.01) para cuantiles

In [0]:
df = loan_df

In [0]:
display(df.describe())

In [0]:
display(df.summary("min", "25%", "50%", "75%", "max"))

In [0]:
df.columns

In [0]:
quantiles = df.approxQuantile('monthly_income', [0.25, 0.5, 0.75], 0.01)
print(f"Cuantiles aproximados para 'monthly_income': {quantiles}")

In [0]:
print(f"Total rows: {df.count()}")

In [0]:
# 3. Detección de nulos y limpieza
from pyspark.sql.functions import isnull, when,sum,col

# Conteo nulos
nul_counts = df.select([sum(when(isnull(c),1).otherwise(0)).alias(c) for c in df.columns])
nul_counts.display()

In [0]:
# Drop filas con loan_amount_requested nulo y llenar income con 0
df_clean = (df
    .dropna(subset=["loan_amount_requested"])
    .fillna({"monthly_income": 0})
    .withColumnRenamed("fraud_flag", "is_fraud")
)
display(df_clean)

In [0]:
from pyspark.sql.functions import when, col, expr

# 4. Tipado y renombrado
df2 = (df_clean
  .withColumnRenamed('application_date', 'app_date')
  .withColumn('is_fraud_2', when(col('is_fraud') == 1, True).otherwise(False))
  .withColumn('loan_amount_requested', expr("try_cast(loan_amount_requested as double)"))
)

df2.limit(10).display()

In [0]:
# 4. SQL vs API de DataFrame
df2.createOrReplaceTempView('loans')
df_sql = spark.sql("SELECT customer_id, loan_amount_requested FROM loans WHERE monthly_income > 50000")

In [0]:
df_sql.display()

In [0]:
df_api = df2.filter(col('monthly_income')>50000).selectExpr('customer_id','loan_amount_requested')

In [0]:
df_sql.display()

In [0]:
df_api.display()

# Databricks Notebook: Módulo 3 - Transformaciones Avanzadas

In [0]:
from pyspark.sql.functions import when, concat, lit, to_date,col

# 1. Columnas derivadas
loan_feat = (df_clean
    .withColumn("risk_score", when(col("loan_amount_requested")/col("monthly_income")>0.5, lit("High")).otherwise(lit("Low")))    
    .withColumn("app_date", to_date(col("application_date"), 'yyyy-MM-dd'))
)

In [0]:
loan_feat.display()

In [0]:
# 2. Joins: sumar amount por cliente y unir
sum_trans = (trans_df
    .groupBy("customer_id")
    .agg(sum("transaction_amount").alias("total_spent"))
)

In [0]:
sum_trans.display()

In [0]:
merged = loan_feat.join(sum_trans, on="customer_id", how="left")
merged.display()

In [0]:
# 3. Window: ranking de préstamos por income
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

w = Window.orderBy(col("monthly_income").desc())
loan_rank = merged.withColumn("rank", row_number().over(w))
loan_rank.display()

In [0]:
trans_df.columns

In [0]:
trans_df.display()

In [0]:
# 4. Trabajar con pivot y unpivot

from pyspark.sql.functions import count

pivot_df = trans_df.groupBy("customer_id").pivot("merchant_category").agg(sum("transaction_amount"))

pivot_df.display()

In [0]:
# Obtener columnas de categorías (sin 'customer_id')
category_columns = [col for col in pivot_df.columns if col != "customer_id"]
print(category_columns)

In [0]:
# Construir expresión stack
n = len(category_columns)
expr = f"stack({n}, " + ", ".join([f"'{c}', `{c}`" for c in category_columns]) + f") as (merchant_category, total_amount)"

# Aplicar unpivot
unpivot_df = pivot_df.selectExpr("customer_id", expr)

display(unpivot_df)

In [0]:
# 5. Trabajar con funciones UDF
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

def risk_score(monthly_income, debt_ratio, loan_amount, emis, employment, dependents):
    if monthly_income is None or debt_ratio is None:
        return "Unknown"

    score = 0

    # Ingreso
    if monthly_income < 20000:
        score += 2
    elif monthly_income < 40000:
        score += 1

    # Ratio deuda
    if debt_ratio > 30:
        score += 2
    elif debt_ratio > 15:
        score += 1

    # Loan grande con ingreso bajo
    if loan_amount is not None and monthly_income is not None and loan_amount > (monthly_income * 12):
        score += 1

    # EMIs altos
    if emis is not None and emis > (monthly_income * 0.3):
        score += 1

    # Empleo inestable
    if employment in ["Unemployed", "Retired"]:
        score += 1

    # Muchos dependientes
    if dependents is not None and int(dependents) >= 3:
        score += 1

    # Clasificación final
    if score >= 5:
        return "High Risk"
    elif score >= 3:
        return "Medium Risk"
    else:
        return "Low Risk"

In [0]:
risk_score_udf = udf(risk_score, StringType())

# 5️⃣ Aplicar UDF
df_with_risk = loan_df.withColumn(
    "risk_category",
    risk_score_udf(
        loan_df["monthly_income"],
        loan_df["debt_to_income_ratio"],
        loan_df["loan_amount_requested"],
        loan_df["existing_emis_monthly"],
        loan_df["employment_status"],
        loan_df["number_of_dependents"]
    )
)

In [0]:
df_with_risk.select(
    "customer_id", "monthly_income", "debt_to_income_ratio",
    "loan_amount_requested", "employment_status", "number_of_dependents",
    "risk_category"
).display()

# Databricks Notebook: Módulo 4 - Perspectivas de Rendimiento y Tuning

### Particiones
¿Por qué importa?

- Controlar particiones optimiza la paralelización y reduce cuellos de botella.

- Usar repartition cuando quieres redistribuir y barajar datos.
 
- Usar coalesce cuando quieres fusionar particiones sin barajar datos (típico en etapas finales).

In [0]:
# 1️⃣ Aumentar particiones con shuffle
df_re = trans_df.repartition(200, "customer_id")
df_re.display()

In [0]:
# 2️⃣ Reducir particiones sin shuffle
df_co = trans_df.coalesce(3)
df_co.display()

In [0]:
# 3️⃣ Detección de skew (desequilibrio)
skew = trans_df.groupBy("customer_id").count().orderBy(col("count").desc())
skew.display()

### Broadcast Join vs Shuffle Join
¿Qué son?

- Broadcast Join: envía un DataFrame pequeño a todos los nodos para evitar shuffle costoso. Ideal si uno de los DataFrames cabe en la RAM de cada executor.

- Shuffle Join: Spark hace shuffle de ambas tablas para repartir claves de join.

In [0]:
from pyspark.sql.functions import broadcast

# Ejemplo: supongamos que stats es pequeño (e.g. loan stats por cliente)
stats = loan_df.groupBy("customer_id").agg({"loan_amount_requested": "avg"})

In [0]:
stats.display()

In [0]:
# 1️⃣ Broadcast Join: rápido si stats es pequeño
df_bc = trans_df.join(broadcast(stats), "customer_id")
df_bc.display()

In [0]:
# 2️⃣ Shuffle Join: normal
df_shuf = trans_df.join(stats, "customer_id")

In [0]:
df_shuf.display()